# Dynamic Time Warping

GOAL: classify time series with the 1-NN algorithm, usign two different approaches to compute the distance between time series: the Euclidean distance and the Dynamic Time Warping (DTW) distance. Compare their performance

## Processing the data

In [12]:
import pandas as pd

data = pd.read_csv('hour.csv', parse_dates = [1])
print(data.shape)
data.head(10)

(17379, 17)


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,7,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,8,2011-01-01,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,9,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,10,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [13]:
ts = data.groupby(data['dteday'].dt.date)['cnt'].agg(lambda x: list(x)).to_frame('counts')
labels = data.groupby(data['dteday'].dt.date)['workingday'].agg('mean').to_frame()

ts['workingday'] = labels
ts.head(20)

,counts,workingday
dteday,,
2011-01-01,"[16, 40, 32, 13, 1, 1, 2, 3, 8, 14, 36, 56, 84...",0
2011-01-02,"[17, 17, 9, 6, 3, 2, 1, 8, 20, 53, 70, 93, 75,...",0
2011-01-03,"[5, 2, 1, 3, 30, 64, 154, 88, 44, 51, 61, 61, ...",1
2011-01-04,"[5, 2, 1, 2, 4, 36, 94, 179, 100, 42, 57, 78, ...",1
2011-01-05,"[6, 6, 2, 2, 3, 33, 88, 195, 115, 57, 46, 79, ...",1
2011-01-06,"[11, 4, 2, 1, 4, 36, 95, 219, 122, 45, 59, 84,...",1
2011-01-07,"[17, 7, 1, 1, 5, 34, 84, 210, 134, 63, 67, 59,...",1
2011-01-08,"[25, 16, 16, 7, 1, 5, 2, 9, 15, 20, 61, 62, 98...",0
2011-01-09,"[25, 12, 11, 4, 1, 1, 1, 6, 10, 19, 49, 49, 83...",0


In [14]:
ts.shape

(731, 2)

In [15]:
X = ts['counts']
y = ts['workingday']

In [18]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=7)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(489,)
(242,)
(489,)
(242,)


/home/mariya/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Implementing the algorithms

In [33]:
import math

def euclid_dist(s1, s2):
    sqddiffs = [(a_i - b_i)**2 for a_i, b_i in zip(s1,s2)]
    return math.sqrt(sum(sqddiffs))

In [20]:
def oneNearestNeighbor(X_train, y_train, test_s, distance):
    min_dist = float('inf')
    prediction = -1
    for i in range(len(y_train)):
        d = distance(X_train[i], test_s)
        if d < min_dist:
            min_dist = d
            prediction = y_train[i]
    return prediction

In [34]:
def classify(X_train, y_train, X_test, y_test, distance):
    correct = 0.0
    for j in range(len(y_test)):
        pred = oneNearestNeighbor(X_train, y_train, X_test[j], distance)
        if pred == y_test[j]:
            correct += 1
    return correct/len(y_test)

In [36]:
accuracy = classify(X_train, y_train, X_test, y_test, euclid_dist)
print(accuracy)

0.6818181818181818


### Implementation of the DTW distance.

In [39]:
def DTWDistance(s1, s2): # returns the DTW distance between two time series s1 and s2
    DTW={}

    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return math.sqrt(DTW[len(s1)-1, len(s2)-1])

In [40]:
accuracy = classify(X_train, y_train, X_test, y_test, DTWDistance) # should take ~3 min to run
print(accuracy)

0.9669421487603306


In [41]:
def trim(row):  # 'trim' a time series by removing elements from it
    tmp = []
    for c in row.counts:
        if c > 50:
            tmp.append(c)
    row.counts = tmp
    return row

varts = ts.apply(trim, axis=1) # apply our trim method on all rows of the ts datarame
varts.head()

,counts,workingday
dteday,,
2011-01-01,"[56, 84, 94, 106, 110, 93, 67]",0
2011-01-02,"[53, 70, 93, 75, 59, 74, 76, 65, 53]",0
2011-01-03,"[64, 154, 88, 51, 61, 61, 77, 72, 76, 157, 157...",1
2011-01-04,"[94, 179, 100, 57, 78, 97, 63, 65, 83, 212, 18...",1
2011-01-05,"[88, 195, 115, 57, 79, 71, 62, 62, 89, 190, 16...",1


In [42]:
X_train, X_test, y_train, y_test = train_test_split(varts['counts'], varts['workingday'], test_size=0.33, random_state=42)

In [43]:
euclid_accuracy = classify(X_train, y_train, X_test, y_test, euclid_dist)
DTW_accuracy = classify(X_train, y_train, X_test, y_test, DTWDistance) 
print(euclid_accuracy)
print(DTW_accuracy)

0.17355371900826447
0.9793388429752066


Conclusion: DTW works good both with trimmed data and original one